In [13]:
import pandas as pd
df = pd.read_csv('Test_Case_dataCleaning.csv')
print(df.head(2))


                                        BODY_CLEANED
0  {\n  "meta": {\n    "messageType": "VehicleDat...
1  {\n  "meta": {\n    "messageType": "VehicleDat...


In [31]:
import pandas as pd
import json

# Load only the first 10 rows
df = pd.read_csv('Test_Case_dataCleaning.csv', nrows=10)
print(df)

                                        BODY_CLEANED
0  {\n  "meta": {\n    "messageType": "VehicleDat...
1  {\n  "meta": {\n    "messageType": "VehicleDat...
2  {\n  "meta": {\n    "messageType": "VehicleDat...
3  {\n  "meta": {\n    "messageType": "VehicleDat...
4  {\n  "meta": {\n    "messageType": "VehicleDat...
5  {\n  "meta": {\n    "messageType": "VehicleDat...
6  {\n  "meta": {\n    "messageType": "VehicleDat...
7  {\n  "meta": {\n    "messageType": "VehicleDat...
8  {\n  "meta": {\n    "messageType": "VehicleDat...
9  {\n  "meta": {\n    "messageType": "VehicleDat...


In [21]:
import pandas as pd

# Load the CSV with only the first row to avoid overload
df = pd.read_csv('Test_Case_dataCleaning.csv', nrows=1)

# Raw escaped JSON string from BODY_CLEANED column
raw_json = df.loc[0, 'BODY_CLEANED']

# Decode it to readable format
print(raw_json.encode('utf-8').decode('unicode_escape'))


{
  "meta": {
    "messageType": "VehicleData",
    "pollerFetchTime": 1751725939315,
    "version": "v2"
  },
  "partitionKey": "df979dc9-5cc6-470f-a105-d3108fd03c19",
  "timestamp": "20250705143219.297",
  "transactionId": "df979dc9-5cc6-470f-a105-d3108fd03c19",
  "vinList": [
    {
      "dischargeWarning": {
        "hazardStatus": 0,
        "tailLampStatus": 0
      },
      "evStatusData": [
        {
          "attribute": "ign3Status",
          "dataType": 1,
          "value": "0"
        }
      ],
      "headLampStatus": {
        "headLamp": 0,
        "leftHighLamp": 0,
        "leftLowLamp": 0,
        "rightHighLamp": 0,
        "rightLowLamp": 0
      },
      "locationTimestamp": {
        "offSet": -7,
        "time": "20250705143213.000"
      },
      "statusData": [
        {
          "attribute": "odometer",
          "dataType": 2,
          "unit": 1,
          "value": "28082.6"
        },
        {
          "attribute": "gasLevel",
          "dataType": 1,

In [23]:
print(df)

                                        BODY_CLEANED
0  {\n  "meta": {\n    "messageType": "VehicleDat...


In [25]:
import pandas as pd
import json

# Extract the JSON string from the first row
raw_json = df.loc[0, 'BODY_CLEANED']

# Clean the JSON string (if needed)
cleaned_json = raw_json.replace('\\n', '').replace('\\"', '"').replace('\\', '')

# Parse the JSON string
parsed_json = json.loads(cleaned_json)

# Extract meta information
poller_fetch_time = parsed_json["meta"].get("pollerFetchTime", None)

# Get the list of vehicles (usually length 1)
vin_list = parsed_json.get("vinList", [])

# Flatten the structure
parsed_data = []

for vin_data in vin_list:
    vin = vin_data.get("vin", None)
    status_timestamp = vin_data.get("statusTimestamp", {}).get("time", None)
    
    for item in vin_data.get("statusData", []):
        parsed_data.append({
            "VIN": vin,
            "statusTimestamp": status_timestamp,
            "attribute": item.get("attribute", None),
            "value": item.get("value", None),
            "pollerFetchTime": poller_fetch_time
        })

# Convert to DataFrame
flattened_df = pd.DataFrame(parsed_data)

# Display the result
print(flattened_df)


             VIN     statusTimestamp          attribute    value  \
0   MASKEDVIN003  20250705143210.000           odometer  28082.6   
1   MASKEDVIN003  20250705143210.000           gasLevel       24   
2   MASKEDVIN003  20250705143210.000    parkingPosition        1   
3   MASKEDVIN003  20250705143210.000    engineOilStatus        0   
4   MASKEDVIN003  20250705143210.000            airTemp        0   
5   MASKEDVIN003  20250705143210.000     breakOilStatus        0   
6   MASKEDVIN003  20250705143210.000          ignStatus        1   
7   MASKEDVIN003  20250705143210.000      accModeStatus        1   
8   MASKEDVIN003  20250705143210.000          batStatus       92   
9   MASKEDVIN003  20250705143210.000  washerFluidStatus        1   
10  MASKEDVIN003  20250705143210.000    traveldirection      268   
11  MASKEDVIN003  20250705143210.000              speed        0   

    pollerFetchTime  
0     1751725939315  
1     1751725939315  
2     1751725939315  
3     1751725939315  
4    

In [29]:
import pandas as pd
import json

# Step 1: Load and clean the first row of JSON
raw_json = df.loc[0, 'BODY_CLEANED']
cleaned_json = raw_json.replace('\\n', '').replace('\\"', '"').replace('\\', '')
parsed_json = json.loads(cleaned_json)

# Step 2: Extract required info
poller_fetch_time = parsed_json["meta"].get("pollerFetchTime", None)
vin_list = parsed_json.get("vinList", [])

# Step 3: Define the attributes you want to keep
wanted_attributes = {"odometer", "gasLevel", "ignStatus", "speed"}

# Step 4: Build the final row
records = []
for vin_data in vin_list:
    flat_row = {
        "VIN": vin_data.get("vin", None),
        "statusTimestamp": vin_data.get("statusTimestamp", {}).get("time", None),
        "pollerFetchTime": poller_fetch_time
    }
    
    # Pull only the desired attributes
    for item in vin_data.get("statusData", []):
        attr = item.get("attribute", None)
        if attr in wanted_attributes:
            flat_row[attr] = item.get("value", None)
    
    records.append(flat_row)

# Step 5: Convert to DataFrame
final_df = pd.DataFrame(records)

# Reorder columns (optional)
final_df = final_df[["VIN", "statusTimestamp", "odometer", "gasLevel", "ignStatus", "speed", "pollerFetchTime"]]

# Display result
print(final_df.to_string(index=False))


         VIN    statusTimestamp odometer gasLevel ignStatus speed  pollerFetchTime
MASKEDVIN003 20250705143210.000  28082.6       24         1     0    1751725939315


In [33]:
print(df)

                                        BODY_CLEANED
0  {\n  "meta": {\n    "messageType": "VehicleDat...
1  {\n  "meta": {\n    "messageType": "VehicleDat...
2  {\n  "meta": {\n    "messageType": "VehicleDat...
3  {\n  "meta": {\n    "messageType": "VehicleDat...
4  {\n  "meta": {\n    "messageType": "VehicleDat...
5  {\n  "meta": {\n    "messageType": "VehicleDat...
6  {\n  "meta": {\n    "messageType": "VehicleDat...
7  {\n  "meta": {\n    "messageType": "VehicleDat...
8  {\n  "meta": {\n    "messageType": "VehicleDat...
9  {\n  "meta": {\n    "messageType": "VehicleDat...


In [35]:
import pandas as pd
import json

# Store processed rows
flattened_data = []

# Iterate through all rows in df (already limited to 10 rows)
for index, row in df.iterrows():
    try:
        raw_json = row['BODY_CLEANED'].strip()
        clean_json = raw_json.replace('\\n', '').replace('\\', '').replace('\"', '"')
        data = json.loads(clean_json)

        pollerFetchTime = data.get("meta", {}).get("pollerFetchTime", None)
        vinList = data.get("vinList", [])

        for vin_entry in vinList:
            vin = vin_entry.get("vin", None)
            statusTimestamp = vin_entry.get("statusTimestamp", {}).get("time", None)
            statusData = vin_entry.get("statusData", [])

            # Initialize row
            row_dict = {
                "VIN": vin,
                "statusTimestamp": statusTimestamp,
                "pollerFetchTime": pollerFetchTime,
                "odometer": None,
                "gasLevel": None,
                "ignStatus": None,
                "speed": None
            }

            for item in statusData:
                attr = item.get("attribute")
                value = item.get("value")
                if attr in row_dict:
                    row_dict[attr] = value

            flattened_data.append(row_dict)

    except Exception as e:
        print(f"Error in row {index}: {e}")

# Create the final DataFrame
final_df = pd.DataFrame(flattened_data)

# Show result without slashes and clean format
print(final_df.to_string(index=False))


         VIN    statusTimestamp  pollerFetchTime odometer gasLevel ignStatus speed
MASKEDVIN003 20250705143210.000    1751725939315  28082.6       24         1     0
MASKEDVIN003 20250708170037.000    1751994040672  28605.6      100         1    28
MASKEDVIN003 20250706181414.000    1751825658365  28443.6       37         0     0
MASKEDVIN003 20250708204314.000    1752007396151  28902.2       72         1    55
MASKEDVIN003 20250705202326.000    1751747010652  28275.5       63         1     0
MASKEDVIN003 20250709183810.000    1752086292389  29296.9       23         0     0
MASKEDVIN003 20250709015822.000    1752026304311  29212.4       33         1    34
MASKEDVIN003 20250709182637.000    1752085600580  29296.1       23         1     6
MASKEDVIN003 20250707011609.000    1751850974094  28516.7       25         1     2
MASKEDVIN003 20250707022311.000    1751854993858  28533.7       22         1     3
